In [122]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score

In [124]:
train = pd.read_csv("/Users/am/Downloads/Practice/playground/train.csv")
train

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517749,517749,highway,4,0.10,70,daylight,foggy,True,True,afternoon,False,False,2,0.32
517750,517750,rural,4,0.47,35,daylight,rainy,True,True,morning,False,False,1,0.26
517751,517751,urban,4,0.62,25,daylight,foggy,False,False,afternoon,False,True,0,0.19
517752,517752,highway,3,0.63,25,night,clear,True,False,afternoon,True,True,3,0.51


In [125]:
X_test = pd.read_csv("/Users/am/Downloads/Practice/playground/test.csv")
X_test

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents
0,517754,highway,2,0.34,45,night,clear,True,True,afternoon,True,True,1
1,517755,urban,3,0.04,45,dim,foggy,True,False,afternoon,True,False,0
2,517756,urban,2,0.59,35,dim,clear,True,False,afternoon,True,True,1
3,517757,rural,4,0.95,35,daylight,rainy,False,False,afternoon,False,False,2
4,517758,highway,2,0.86,35,daylight,clear,True,False,evening,False,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172580,690334,rural,2,0.01,45,dim,rainy,False,False,afternoon,True,True,2
172581,690335,rural,1,0.74,70,daylight,foggy,False,True,afternoon,False,False,2
172582,690336,urban,2,0.14,70,dim,clear,False,False,evening,True,True,1
172583,690337,urban,1,0.09,45,daylight,foggy,True,True,morning,False,True,0


In [126]:
Y_train = train['accident_risk']
X_train = train.drop('accident_risk', axis = 1)

In [127]:
print("Training shape:", X_train.shape)
print("Test shape:", X_test.shape)

Training shape: (517754, 13)
Test shape: (172585, 13)


In [129]:
from sklearn.model_selection import train_test_split

X_train_split, X_train_val, Y_train_split, Y_train_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42 )

In [130]:
# co_mtx = X_train_split.corr(numeric_only=True)
# sns.heatmap(co_mtx)

In [136]:
categorical_features = ['road_type', 'lighting','weather','road_signs_present', 
                        'public_road','time_of_day', 'holiday','school_season']

numeric_features = ['num_lanes', 'curvature', 'speed_limit', 'num_reported_accidents']

X_train_split_cat = X_train_split[categorical_features]
X_train_split_num = X_train_split[numeric_features]

X_train_val_cat = X_train_val[categorical_features]
X_train_val_num = X_train_val[numeric_features]

X_test_cat = X_test[categorical_features]
X_test_num = X_test[numeric_features]



In [137]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # sparse=False to get a DataFrame/array
encoder.fit(X_train_split_cat)

X_train_split_cat_encoded = encoder.transform(X_train_split_cat)
X_train_val_cat_encoded = encoder.transform(X_train_val_cat)
X_test_encoded = encoder.transform(X_test_cat)

In [139]:
# Combine numeric + encoded categorical
X_train_split_final = np.hstack([X_train_split_num.values, X_train_split_cat_encoded])
X_train_val_final = np.hstack([X_train_val_num.values, X_train_val_cat_encoded])
X_test_final = np.hstack([X_test_num.values, X_test_encoded])

In [140]:
########## Random Forest Regression ########## 

In [141]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

rf.fit(X_train_split_final, Y_train_split)


RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=42)

In [145]:
#Predict
y_pred_train_val_rf = rf.predict(X_train_val_final)
y_pred_train_val_rf

array([0.12911824, 0.33139707, 0.25105742, ..., 0.40850726, 0.1406238 ,
       0.1556587 ])

In [146]:
# rmse = np.sqrt(mean_squared_error(Y_train_val, y_pred_train_val_rf))
# r2 = r2_score(Y_train_val, y_pred_train_val_rf)

# print("Root Mean Squared Error:", rmse)
# print("R² Score:", r2)

Root Mean Squared Error: 0.056309668722782966
R² Score: 0.8851674459011593


In [29]:
########## hyperparameter tuning ##########

In [31]:
# rf = RandomForestRegressor(n_estimators=900, max_depth=10, min_samples_leaf=5,random_state=42, n_jobs=-1)

# rf.fit(X_train_split_final, Y_train_split)

# rmse = np.sqrt(mean_squared_error(Y_train_val, y_pred_train_val_rf))
# r2 = r2_score(Y_train_val, y_pred_train_val_rf)

# print("Root Mean Squared Error:", rmse)
# print("R² Score:", r2)

Root Mean Squared Error: 0.056309668722782966
R² Score: 0.8851674459011593


In [32]:
########## Apply k-fold 

In [33]:
# X_train_cat = X_train[categorical_features]
# X_train_num = X_train[numeric_features]

# encoder.fit(X_train_cat)

# X_train_cat_encoded = encoder.transform(X_train_cat)

# X_train_full = np.hstack([X_train_num.values, X_train_cat_encoded])


In [37]:
# long running time

In [39]:
# from sklearn.model_selection import KFold

# test_preds = np.zeros(len(X_test_final))

# fold_rmse = []

# kf = KFold(n_splits=3, shuffle=True, random_state=42)

# for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
#     print(f"\n===== Fold {fold+1} =====")

#     X_train_rfkf, X_val_rfkf = X_train_full[train_idx], X_train_full[val_idx]
#     y_train_rfkf, y_val_rfkf = Y_train[train_idx], Y_train[val_idx]
    
#     rf.fit(X_train_rfkf, y_train_rfkf)   
    
#     # Predict on validation set
#     y_pred_rfkf = rf.predict(X_val_rfkf)
    
#     rmse_rfkf = np.sqrt(mean_squared_error(y_val_rfkf, y_pred_rfkf))
#     fold_rmse.append(rmse_rfkf)
#     print(f"Fold {fold+1} RMSE: {rmse_rfkf:.8f}")

#     test_preds += rf.predict(X_test_final) / kf.n_splits

# print(f"\nAverage RMSE across folds: {np.mean(fold_rmse):.8f}")


===== Fold 1 =====
Fold 1 RMSE: 0.05632125

===== Fold 2 =====
Fold 2 RMSE: 0.05618032

===== Fold 3 =====
Fold 3 RMSE: 0.05604019

Average RMSE across folds: 0.05618058


In [43]:
# y_pred_test_rf = test_preds
# y_pred_test_rf

array([0.29895225, 0.1217086 , 0.1777452 , ..., 0.24671873, 0.12916061,
       0.48657885])

In [ ]:
########## Gradient Boosting Regression ########## 

In [150]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=500, learning_rate=0.1, max_depth=4,            
    min_samples_leaf=3, random_state=42)

gbr.fit(X_train_split_final, Y_train_split)

GradientBoostingRegressor(max_depth=4, min_samples_leaf=3, n_estimators=500,
                          random_state=42)

In [151]:
y_pred_train_val_gbr = gbr.predict(X_train_val_final)

rmse_gbr = np.sqrt(mean_squared_error(Y_train_val, y_pred_train_val_gbr))
r2_gbr = r2_score(Y_train_val, y_pred_train_val_gbr)

print("Root Mean Squared Error:", rmse_gbr)
print("R² Score:", r2_gbr)

Root Mean Squared Error: 0.05640946985821051
R² Score: 0.8847600353959664


In [152]:
########## Apply k-fold 

In [153]:
X_train_cat = X_train[categorical_features]
X_train_num = X_train[numeric_features]

encoder.fit(X_train_cat)

X_train_cat_encoded = encoder.transform(X_train_cat)

X_train_full = np.hstack([X_train_num.values, X_train_cat_encoded])

In [154]:
from sklearn.model_selection import KFold

test_preds_grb = np.zeros(len(X_test_final))

fold_rmse = []

kf = KFold(n_splits=3, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
    print(f"\n===== Fold {fold+1} =====")

    X_train_gbrkf, X_val_gbrkf = X_train_full[train_idx], X_train_full[val_idx]
    y_train_gbrkf, y_val_gbrkf = Y_train[train_idx], Y_train[val_idx]
    
    gbr.fit(X_train_gbrkf, y_train_gbrkf)   
    
    # Predict on validation set
    y_pred_gbrkf = gbr.predict(X_val_gbrkf)
    
    rmse_gbrkf = np.sqrt(mean_squared_error(y_val_gbrkf, y_pred_gbrkf))
    fold_rmse.append(rmse_gbrkf)
    print(f"Fold {fold+1} RMSE: {rmse_gbrkf:.8f}")

    test_preds_grb += gbr.predict(X_test_final) / kf.n_splits

print(f"\nAverage RMSE across folds: {np.mean(fold_rmse):.8f}")


===== Fold 1 =====
Fold 1 RMSE: 0.05643245

===== Fold 2 =====
Fold 2 RMSE: 0.05627100

===== Fold 3 =====
Fold 3 RMSE: 0.05614752

Average RMSE across folds: 0.05628366


In [155]:
# Predict
y_pred_test_rf = test_preds_grb
y_pred_test_rf

array([0.29167794, 0.12403061, 0.19286005, ..., 0.25199063, 0.12867109,
       0.49181656])

In [57]:
########## XGBoost ##########

In [160]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [161]:
import xgboost as xgb

xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

xgb.fit(X_train_split_final,Y_train_split)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [162]:
y_pred_xgb = xgb.predict(X_train_val_final)

rmse_xgb = np.sqrt(mean_squared_error(Y_train_val, y_pred_xgb))
r2_xgb = r2_score(Y_train_val, y_pred_xgb)

print("Root Mean Squared Error:", rmse_xgb)
print("R² Score:", r2_xgb)

Root Mean Squared Error: 0.05702068250426873
R² Score: 0.8822491902634115


In [168]:
########## Apply k-fold 
# X_train_cat = X_train[categorical_features]
# X_train_num = X_train[numeric_features]

# encoder.fit(X_train_cat)

# X_train_cat_encoded = encoder.transform(X_train_cat)

# X_train_full = np.hstack([X_train_num.values, X_train_cat_encoded])

In [170]:
test_preds_xgb = np.zeros(len(X_test_final))

fold_rmse = []

kf = KFold(n_splits = 3, shuffle = True, random_state = 42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
    print(f"\n===== Fold {fold+1} =====")

    X_train_xgbkf, X_val_xgbkf = X_train_full[train_idx], X_train_full[val_idx]
    y_train_xgbkf, y_val_xgbkf = Y_train[train_idx], Y_train[val_idx]

    xgb.fit(X_train_xgbkf,y_train_xgbkf)

    # predict on validation set
    y_pred_xbgkf = xgb.predict(X_val_xgbkf)

    rmse_xgbkf = np.sqrt(mean_squared_error(y_val_xgbkf, y_pred_xbgkf))
    fold_rmse.append(rmse_xgbkf)
    print(f"Fold {fold+1} RMSE: {rmse_xgbkf:.8f}")

    test_preds_xgb += xgb.predict(X_test_final) / kf.n_splits

print(f"\nAverage RMSE across folds: {np.mean(fold_rmse):.8f}")


===== Fold 1 =====
Fold 1 RMSE: 0.05699451

===== Fold 2 =====
Fold 2 RMSE: 0.05689119

===== Fold 3 =====
Fold 3 RMSE: 0.05672435

Average RMSE across folds: 0.05687002


In [171]:
# Predict
y_pred_test_xgb = test_preds_xgb
y_pred_test_xgb

array([0.28631081, 0.12760276, 0.19316244, ..., 0.24968368, 0.13192366,
       0.48258881])

In [172]:
########## Lightgbm ##########

In [176]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [177]:
import lightgbm as lgb

lgb = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05, max_depth=-1, subsample=0.8,
        colsample_bytree=0.8, random_state=42)

lgb.fit(X_train_split_final,Y_train_split)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007722 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 187
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 24
[LightGBM] [Info] Start training from score 0.352605


LGBMRegressor(colsample_bytree=0.8, learning_rate=0.05, n_estimators=1000,
              random_state=42, subsample=0.8)

In [180]:
y_pred_lgb = lgb.predict(X_train_val_final)

rmse_lgb = np.sqrt(mean_squared_error(Y_train_val, y_pred_lgb))
r2_lgb = r2_score(Y_train_val, y_pred_lgb)

print("Root Mean Squared Error:", rmse_lgb)
print("R² Score:", r2_lgb)

Root Mean Squared Error: 0.05627426601851341
R² Score: 0.8853117942897358


In [181]:
y_pred_test_lgb = lgb.predict(X_test_final)
y_pred_test_lgb

array([0.28919456, 0.12194197, 0.18513125, ..., 0.25284846, 0.12688031,
       0.48783072])

In [182]:
########## Apply k-fold 

In [183]:
test_preds_lgb = np.zeros(len(X_test_final))

fold_rmse = []

kf = KFold(n_splits = 3, shuffle = True, random_state = 42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
    print(f"\n===fold {fold +1} ====")
    
    X_train_split_lgbkf, X_train_val_lgbkf = X_train_full[train_idx], X_train_full[val_idx]
    Y_train_split_lgbkf, Y_train_val_lgbkf = Y_train[train_idx], Y_train[val_idx]

    lgb.fit(X_train_split_lgbkf,Y_train_split_lgbkf)

    y_pred_val = lgb.predict(X_train_val_lgbkf)

    rmse_lgbkf = np.sqrt(mean_squared_error(y_pred_val,Y_train_val_lgbkf))
    fold_rmse.append(rmse_lgbkf)
    print(f"==== fold {fold +1} RMSE : {rmse_lgbkf:.8f}")

    test_preds_lgb += lgb.predict(X_test_final) / kf.n_splits

print(f"\nAverage RMSE across folds: {np.mean(fold_rmse):.8f}")


===fold 1 ====
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006281 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 185
[LightGBM] [Info] Number of data points in the train set: 345169, number of used features: 24
[LightGBM] [Info] Start training from score 0.352438
==== fold 1 RMSE : 0.05631570

===fold 2 ====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.120128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 345169, number of used features: 24
[LightGBM] [Info] Start training from score 0.352326
==== fold 2 RMSE : 0.05613730

===fold 3 ====
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010937 seconds.
You can set `force_row_wise=true` to rem

In [187]:
### Base Prediction

# grb_test_pred = grb.predict(X_test_final)
# xgb_test_pred = xgb.predict(X_test_final)
# lgb_test_pred = lgb.predict(X_test_final)

# grb_val_pred = grb.predict(X_train_val_final)
# xgb_val_pred = xgb.predict(X_train_val_final)
# lgb_val_pred = lgb.predict(X_train_val_final)



In [188]:
# from sklearn.isotonic import IsotonicRegression

# # Manual Stacking w/o KFold

# def calibrate_predictions(y_true, y_pred, y_test_pred):
#     calibrator = IsotonicRegression(out_of_bounds='clip', y_min=0, y_max=1)
#     calibrator.fit(y_pred, y_true)
#     calibrated_pred = calibrator.predict(y_test_pred)
#     return np.clip(calibrated_pred, 0, 1)

# # Calibrate validation predictions
# xgb_val_calibrated = calibrate_predictions(y_val, xgb_val_pred, xgb_val_pred)
# lgb_val_calibrated = calibrate_predictions(y_val, lgb_val_pred, lgb_val_pred)
# gbr_val_calibrated = calibrate_predictions(y_val, gbr_val_pred, gbr_val_pred)

# meta_x = np.column_stack([xgb_val_calibrated,lgb_val_calibrated,gbr_val_calibrated])
# meta_y = Y_train_val

# meta_model = Ridge()
# meta_model.fit(meta_x,meta_y)

# xgb_test_pred = xgb.predict(X_test_final)
# lgb_test_pred = lgb.predict(X_test_final)
# cat_test_pred = cat.predict(X_test_final)

# xgb_test_calibrated = calibrate_predictions(y_val, xgb_val_pred, y_pred_test_xgb)
# lgb_test_calibrated = calibrate_predictions(y_val, lgb_val_pred, y_pred_test_lgb)
# gbr_test_calibrated = calibrate_predictions(y_val, gbr_val_pred, y_pred_test_gbr)

# meta_test_X = np.column_stack([xgb_test_calibrated, lgb_test_calibrated, cat_test_calibrated])
# final_pred = meta_model.predict(meta_test_X)

In [190]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

xgb_oof = np.zeros(len(X_train_full))
lgb_oof = np.zeros(len(X_train_full))
gbr_oof = np.zeros(len(X_train_full))

xgb_test_preds = np.zeros((len(X_test_final), 5))
lgb_test_preds = np.zeros((len(X_test_final), 5))
gbr_test_preds = np.zeros((len(X_test_final), 5))

for i, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
    X_tr, X_val = X_train_full[train_idx], X_train_full[val_idx]
    y_tr, y_val = Y_train[train_idx], Y_train[val_idx]
    
    # Train models on each fold
    xgb.fit(X_tr, y_tr)
    lgb.fit(X_tr, y_tr)
    gbr.fit(X_tr, y_tr)
    
    # Out-of-fold predictions (on unseen data)
    xgb_oof[val_idx] = xgb.predict(X_val)
    lgb_oof[val_idx] = lgb.predict(X_val)
    gbr_oof[val_idx] = gbr.predict(X_val)
    
    # Predict on test set
    xgb_test_preds[:, i] = xgb.predict(X_test_final)
    lgb_test_preds[:, i] = lgb.predict(X_test_final)
    gbr_test_preds[:, i] = gbr.predict(X_test_final)

# Average test predictions across folds
xgb_test_final = xgb_test_preds.mean(axis=1)
lgb_test_final = lgb_test_preds.mean(axis=1)
gbr_test_final = gbr_test_preds.mean(axis=1)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009390 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 186
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 24
[LightGBM] [Info] Start training from score 0.352605
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011618 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 186
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 24
[LightGBM] [Info] Start training from score 0.352104
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007208 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

In [211]:
from sklearn.linear_model import Ridge
meta_X = np.column_stack([xgb_oof, lgb_oof, gbr_oof])
meta_model = Ridge()
meta_model.fit(meta_X, Y_train)

meta_X_pred = meta_model.predict(meta_X[val_idx])
rmse_stacked = np.sqrt(mean_squared_error(meta_X_pred, y_val))

meta_test_X = np.column_stack([xgb_test_final, lgb_test_final, gbr_test_final])
final_pred = meta_model.predict(meta_test_X)
final_pred

array([0.29013889, 0.12390666, 0.18775118, ..., 0.25194352, 0.12802657,
       0.4904138 ])

In [213]:
rmse_stacked

0.05594096090178955

In [221]:
submission = pd.DataFrame({
    'id': X_test['id'],        # IDs from test set
    'accident_risk': final_pred  # Your predicted values
})

submission

,id,accident_risk
0,517754,0.290139
1,517755,0.123907
2,517756,0.187751
3,517757,0.316839
4,517758,0.401094
...,...,...
172580,690334,0.108480
172581,690335,0.516009
172582,690336,0.251944
172583,690337,0.128027


In [ ]:
# easy+fast

# from sklearn.ensemble import StackingRegressor
# from sklearn.linear_model import Ridge

# base_models = [
#     ('gbr', GradientBoostingRegressor()),
#     ('xgb', XGBRegressor()),
#     ('lgb', LGBMRegressor())
# ]

# meta_model = Ridge()  # The model that learns how to combine base models

# stack = StackingRegressor(estimators=base_models, final_estimator=meta_model)
# stack.fit(X_train, y_train)


In [170]:
########## 
#### Random Forest w/o k-fold : Root Mean Squared Error: 0.056309668722782966  R² Score: 0.8851674459011593
#### Random Forest w/ k-fold  : Average RMSE across folds: 0.05618058

#### Gradient Boosting w/o k-fold : Root Mean Squared Error: 0.05640946985821051  R² Score: 0.8847600353959664
#### Gradient Boosting w/ k-fold : Average RMSE across folds: 0.05628366

#### XGBoosting w/o k-fold : Root Mean Squared Error: 0.05702068250426873  R² Score: 0.8822491902634115
#### XGBoosting w/ k-fold : Average RMSE across folds: 0.05687002

########## Random Forest w/ k-fold is the best

In [172]:
########## ANSWER ##########